## Dependencies

In [ ]:
# Install the nr-openai-observability library
!pip install -qqq git+https://github.com/newrelic/nr-openai-observability@staging

## Variables

In [ ]:
# Common environment variables for configuring New Relic Python Agent
# Only run this section if you did not update these values in your newrelic.ini file
# or in your Jupyter environment.

import os

os.environ["NEW_RELIC_CUSTOM_INSIGHTS_EVENTS_MAX_ATTRIBUTE_VALUE"] = "4095"

# Apply recommended settings
os.environ["NEW_RELIC_APPLICATION_LOGGING_ENABLED"] = "true"
os.environ["NEW_RELIC_APPLICATION_LOGGING_FORWARDING_ENABLED"] = "true"
os.environ["NEW_RELIC_DISTRIBUTED_TRACING_ENABLED"] = "true"


os.environ["NEW_RELIC_APP_NAME"] = "openai-example"

## Secrets

### [Get your License key](https://one.newrelic.com/launcher/api-keys-ui.api-keys-launcher) (also referenced as ingest - license) and set it as environment variable: NEW_RELIC_LICENSE_KEY. [Click here](https://docs.newrelic.com/docs/apis/intro-apis/new-relic-api-keys/#license-key) for more details and instructions.

In [ ]:
from getpass import getpass

os.environ["NEW_RELIC_LICENSE_KEY"] = getpass("Please enter your New Relic NEW_RELIC_LICENSE_KEY:")

### OpenAI API key can be created [here](https://platform.openai.com/api-keys).

In [ ]:
os.environ["OPENAI_API_KEY"] = getpass("Please enter your OPENAI_API_KEY:")

In [ ]:
# Initialize New Relic Python Agent
import newrelic.agent

newrelic.agent.initialize()
newrelic.agent.register_application(timeout=10)

In [ ]:
# Initialize New Relic additional OpenAI Monitoring
# This step is only needed for the Limited Preview. This instrumentation
# will be available directly in the New Relic Agent for GA.
#
# For this specific example, this block is *NOT* needed. This example
# is using the OpenAI 1.x APIs and for that instrumentation, only
# the New Relic Python Agent is needed.
#
#from nr_openai_observability import monitor

#monitor.initialization()

In [ ]:
# When not utilizing a web framework, we need a python function that can be
# decorated so that the New Relic Python Agent will instrument the method call.

@newrelic.agent.background_task()
@newrelic.agent.function_trace(name="openai")
def invoke_openai():
    from openai import OpenAI

    # prompt is credited to https://prompts.chat/#act-as-a-math-teacher
    prompt = """I want you to act as a math teacher.
    I will provide some mathematical equations or concepts,
    and it will be your job to explain them in easy-to-understand terms.
    This could include providing step-by-step instructions for solving a problem,
    demonstrating various techniques with visuals or suggesting online
    resources for further study."""
    user_request = (
        'My first request is “I need help understanding how probability works."'
    )

    client = OpenAI()
    completion = client.chat.completions.create(
        model="gpt-4",
        max_tokens=500,
        messages=[
            {
                "role": "system",
                "content": prompt,
            },
            {
                "role": "user",
                "content": user_request,
            },
        ],
    )
    print(completion.choices[0].message.content)


In [ ]:
# Make a call to an LLM to generate metrics for NewRelic connection
# Invoking OpenAI for this question will take a little while as it is
# running in a synchronous or blocking mode. Just give it a bit of time
# to finish.

import os

import openai

if os.getenv("OPENAI_API_KEY") is None:
    print("You must set your OPENAI_API_KEY environment variable")
else:
    openai.api_key = os.getenv("OPENAI_API_KEY")
    invoke_openai()


In [ ]:
# Allow the New Relic agent to send final messages as part of shutdown
# The agent by default can send data up to a minute later
newrelic.agent.shutdown_agent(60)

## [Go here](https://one.newrelic.com/nr1-core?filters=%28domain%20%3D%20%27APM%27%20AND%20type%20%3D%20%27APPLICATION%27%29&state=c8e6f1ba-a8e5-b72c-374b-2c28c4e21ec8) to see your application in New Relic